In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

### Jeu de données des espèces en présence only

In [2]:
species_po = pd.read_csv('../../../Data_TER/GLC24-PO-metadata-train.csv')
species_po['speciesId'] = species_po['speciesId'].astype(int)

In [3]:
species_po.head()

,year,month,day,lat,lon,geoUncertaintyInM,taxonRank,date,dayOfYear,speciesId,surveyId
0,2019,5.0,5.0,43.74605,1.573057,6.0,SPECIES,2019-05-05,125,3383,1
1,2021,3.0,17.0,42.12559,0.314948,5.0,SPECIES,2021-03-17,76,1152,2
2,2021,6.0,5.0,48.29520,-0.934518,24.9,SPECIES,2021-06-05,156,6772,3
3,2021,6.0,9.0,53.63367,-2.644535,8.0,SPECIES,2021-06-09,160,3318,4
4,2021,4.0,1.0,49.79471,7.925086,15.0,SPECIES,2021-04-01,91,3374,5


#### Regrouper tous les surveyId par espèce

In [4]:
species_patch = species_po.groupby('speciesId')['surveyId'].agg(list).reset_index()
species_patch

,speciesId,surveyId
0,0,"[35695, 44444, 57757, 92365, 96301, 175296, 18..."
1,1,[1842284]
2,2,"[66890, 82411, 83701, 89358, 97349, 132719, 15..."
3,4,[2226187]
4,5,"[70973, 125823, 223662, 258664, 631539, 730552..."
...,...,...
9704,11248,"[394179, 1548873, 1660413, 1810688, 2167375, 2..."
9705,11250,"[81665, 204058, 305059, 318984, 1064949, 13599..."
9706,11252,"[26072, 310032, 528646, 681664, 1008167, 14557..."
9707,11253,"[166928, 189851, 204294, 443138, 500287, 53266..."


### Jeu de données abiotiques po

In [5]:
po_bio_ave = pd.read_csv('../../../Data_TER/GLC24-PO-train-bioclimatic-average.csv')
po_elev = pd.read_csv('../../../Data_TER/GLC24-PO-train-elevation.csv')
po_human = pd.read_csv('../../../Data_TER/GLC24-PO-train-human-footprint.csv')
po_land = pd.read_csv('../../../Data_TER/GLC24-PO-train-landcover.csv')
po_soilgrid = pd.read_csv('../../../Data_TER/GLC24-PO-train-soilgrids.csv')

In [6]:
abio_po = po_bio_ave.merge(po_elev, on='surveyId')\
        .merge(po_human, on='surveyId')\
                .merge(po_land, on='surveyId')\
                    .merge(po_soilgrid, on='surveyId')

In [22]:
abio_po = abio_po.replace([np.inf], np.nan)
abio_po = abio_po.replace([-np.inf], np.nan)
#On remplace les NA par la moyenne pour la rf
abio_po = abio_po.fillna(abio_po.mean())

In [8]:
abio_po.head()

,surveyId,Bio1,Bio2,Bio3,Bio4,Bio5,Bio6,Bio7,Bio8,Bio9,...,LandCover,Soilgrid-bdod,Soilgrid-cec,Soilgrid-cfvo,Soilgrid-clay,Soilgrid-nitrogen,Soilgrid-phh2o,Soilgrid-sand,Soilgrid-silt,Soilgrid-soc
0,1,2865,88,3,5756,2999,2752,247,2888,2938,...,12.0,145.0,212.0,130.0,303.0,134.0,73.0,306.0,389.0,122.0
1,2,2865,116,3,6844,3028,2721,307,2869,2795,...,17.0,134.0,213.0,189.0,302.0,184.0,74.0,294.0,403.0,223.0
2,3,2842,71,3,4857,2954,2752,202,2814,2906,...,9.0,127.0,211.0,109.0,187.0,257.0,59.0,132.0,681.0,301.0
3,4,2829,59,3,4108,2918,2747,171,2807,2843,...,9.0,103.0,212.0,123.0,216.0,466.0,56.0,429.0,355.0,1044.0
4,5,2833,76,3,6506,2970,2718,252,2901,2791,...,14.0,138.0,195.0,131.0,310.0,198.0,73.0,208.0,480.0,208.0


### Jeu de données présence d'une espèce selon surveyId

In [13]:
species_patch['speciesId'].iloc[1999]

2320

In [9]:
len(species_patch)

9709

In [15]:
presence = []

for i in range(1999, 4000):
    survey_id = species_patch['surveyId'].iloc[i]
    species_id = species_patch['speciesId'].iloc[i]
    # Pour chaque espèce on stock 0 ou 1 pour chaque surveyId
    presence_espece = abio_po['surveyId'].isin(survey_id).astype(int)
    # On rassemble les présences de toutes les espèces dans une liste
    presence.append((str(species_id), presence_espece))
    print(i)

1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085
2086
2087
2088
2089
2090
2091
2092
2093
2094
2095
2096
2097
2098
2099
2100
2101
2102
2103
2104
2105
2106
2107
2108
2109
2110
2111
2112
2113
2114
2115
2116
2117
2118
2119
2120
2121
2122
2123
2124
2125
2126
2127
2128
2129
2130
2131
2132
2133
2134
2135
2136
2137
2138
2139
2140
2141
2142
2143
2144
2145
2146
2147
2148
2149
2150
2151
2152
2153
2154
2155
2156
2157
2158
2159
2160
2161
2162
2163
2164
2165
2166
2167
2168
2169
2170
2171
2172
2173
2174
2175
2176
2177
2178
2179
2180
2181
2182
2183
2184
2185
2186
2187
2188
2189
2190
2191
2192
2193
2194
2195
2196
2197
2198


3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704
3705
3706
3707
3708
3709
3710
3711
3712
3713
3714
3715
3716
3717
3718
3719
3720
3721
3722
3723
3724
3725
3726
3727
3728
3729
3730
3731
3732
3733
3734
3735
3736
3737
3738
3739
3740
3741
3742
3743
3744
3745
3746
3747
3748
3749
3750
3751
3752
3753
3754
3755
3756
3757
3758
3759
3760
3761
3762
3763
3764
3765
3766
3767
3768
3769
3770
3771
3772
3773
3774
3775
3776
3777
3778
3779
3780
3781
3782
3783
3784
3785
3786
3787
3788
3789
3790
3791
3792
3793
3794
3795
3796
3797
3798
3799
3800
3801
3802
3803
3804
3805
3806
3807
3808
3809
3810
3811
3812
3813
3814
3815
3816
3817
3818
3819
3820
3821
3822
3823
3824
3825
3826
3827
3828
3829
3830
3831
3832
3833
3834
3835
3836
3837


In [20]:
presence[2000]

('4648',
 0          0
 1          0
 2          0
 3          0
 4          0
           ..
 3846449    0
 3846450    0
 3846451    0
 3846452    0
 3846453    0
 Name: surveyId, Length: 3846454, dtype: int32)

#### Données Test

In [17]:
test_bio_ave = pd.read_csv('../../Data/Data_2024/Environnemental_Data/test/GLC24-PA-test-bioclimatic-average.csv')
test_elev = pd.read_csv('../../Data/Data_2024/Environnemental_Data/test/GLC24-PA-test-elevation.csv')
test_human = pd.read_csv('../../Data/Data_2024/Environnemental_Data/test/GLC24-PA-test-human-footprint.csv')
test_land = pd.read_csv('../../Data/Data_2024/Environnemental_Data/test/GLC24-PA-test-landcover.csv')
test_soilgrid = pd.read_csv('../../Data/Data_2024/Environnemental_Data/test/GLC24-PA-test-soilgrids.csv')

abiotest = test_bio_ave.merge(test_elev, on='surveyId')\
        .merge(test_human, on='surveyId')\
                .merge(test_land, on='surveyId')\
                    .merge(test_soilgrid, on='surveyId')
                    
abiotest

,surveyId,Bio1,Bio2,Bio3,Bio4,Bio5,Bio6,Bio7,Bio8,Bio9,...,LandCover,Soilgrid-bdod,Soilgrid-cec,Soilgrid-cfvo,Soilgrid-clay,Soilgrid-nitrogen,Soilgrid-phh2o,Soilgrid-sand,Soilgrid-silt,Soilgrid-soc
0,642,2814,57,2,5717,2934,2720,214,2859,2767,...,9.0,114.0,229.0,96.0,95.0,389.0,48.0,634.0,270.0,891.0
1,1792,2826,104,3,7607,2975,2666,309,2922,2780,...,10.0,126.0,204.0,156.0,238.0,259.0,68.0,349.0,411.0,311.0
2,3256,2735,93,3,6168,2876,2611,265,2755,2669,...,10.0,109.0,246.0,193.0,210.0,547.0,59.0,370.0,419.0,717.0
3,3855,2739,95,3,7129,2880,2583,297,2811,2661,...,10.0,112.0,252.0,195.0,189.0,489.0,60.0,375.0,435.0,571.0
4,4889,2809,62,2,5603,2928,2715,213,2854,2797,...,5.0,113.0,146.0,86.0,96.0,220.0,53.0,751.0,153.0,421.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4711,3915838,2835,71,3,5575,2958,2735,223,2802,2795,...,9.0,121.0,132.0,71.0,120.0,411.0,57.0,636.0,243.0,283.0
4712,3916502,2836,44,2,4844,2931,2752,179,2849,2794,...,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4713,3917793,2817,80,2,6979,2958,2687,271,2907,2727,...,4.0,119.0,220.0,173.0,261.0,272.0,54.0,294.0,444.0,384.0
4714,3918865,2816,46,2,6023,2933,2721,212,2896,2765,...,10.0,129.0,219.0,80.0,170.0,323.0,59.0,568.0,261.0,386.0


In [23]:
abiotest = abiotest.replace([np.inf], np.nan)
abiotest = abiotest.replace([-np.inf], np.nan)
#On remplace les NA par la moyenne pour la rf
abiotest = abiotest.fillna(abio_po.mean())

### Random Forest

In [ ]:
%%time
proba_predict={}

X_test = abiotest.iloc[:,1:].values
X_train = abio_po.iloc[:,1:].values

model = RandomForestClassifier(n_estimators=5)

for i in range(len(presence)-1):
    Y_train = presence[i][1]
    model.fit(X_train, Y_train)
    Y_pred = model.predict_proba(X_test)[:,1]
    species_name = presence[i][0]
    proba_predict[species_name] = Y_pred
    print(i)

0


### On additione ces probas avec celle des presences / absences.

In [18]:
po_predictions = pd.DataFrame(proba_predict)
po_predictions

,0,1,2,4,5,6,8,9,12,14,...,2309,2311,2312,2313,2314,2315,2316,2317,2318,2319
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4711,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4712,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4713,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4714,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
po_test = pd.DataFrame(np.ones((4716, 3)), columns=['0', '2', '3'])
po_test


,0,2,3
0,1.0,1.0,1.0
1,1.0,1.0,1.0
2,1.0,1.0,1.0
3,1.0,1.0,1.0
4,1.0,1.0,1.0
...,...,...,...
4711,1.0,1.0,1.0
4712,1.0,1.0,1.0
4713,1.0,1.0,1.0
4714,1.0,1.0,1.0


In [45]:
df = pd.concat([po_predictions, po_test], axis=1).groupby(level=0, axis=1).sum()

C:\Users\mbrei\AppData\Local\Temp\ipykernel_16632\2640747285.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df = pd.concat([po_predictions, po_test], axis=1).groupby(level=0, axis=1).sum()
